In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
import warnings

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [2]:
def remove_all_zero_col(data):
    data = data.copy()
    for col in data.columns:
        if (data[col] == 0).all():
            data.drop(col, axis=1, inplace=True)
    return data

In [3]:
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [4]:
data = pd.read_csv("source/Ligand_candidate.csv")
Substrate_name = data.drop(columns=['Ligand_name', 'Ligand_smiles', 'Substrate_smiles', 'Yield'])
Ligand_name = data.drop(columns=['Substrate_name', 'Ligand_smiles', 'Substrate_smiles', 'Yield'])
df_lig = pd.DataFrame(data['Ligand_smiles'], columns=['Ligand_smiles'])
df_sub = pd.DataFrame(data['Substrate_smiles'], columns=['Substrate_smiles'])
Ligand_smiles = data['Ligand_smiles']
Yields = data['Yield']

print(Substrate_name)
print(Ligand_name)
print(Yields)
print(df_lig)
print(df_sub)

                 Substrate_name
0                  benzophenone
1     4,4'-dimethylbenzophenone
2    4,4'-dimethoxybenzophenone
3     4,4'-difluorobenzophenone
4                  acetophenone
..                          ...
367             alpha-tetralone
368             alpha-tetralone
369             alpha-tetralone
370             alpha-tetralone
371             alpha-tetralone

[372 rows x 1 columns]
    Ligand_name
0            L1
1            L1
2            L1
3            L1
4            L1
..          ...
367         L98
368         L99
369        L100
370        L101
371        L102

[372 rows x 1 columns]
0      20.0
1       9.0
2       7.0
3      18.0
4      34.0
       ... 
367     NaN
368     NaN
369     NaN
370     NaN
371     NaN
Name: Yield, Length: 372, dtype: float64
                                         Ligand_smiles
0    P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=CC=CC=C3)C4=C...
1    P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=CC=CC=C3)C4=C...
2    P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=

In [5]:
# RDKit, Ligand
PandasTools.AddMoleculeColumnToFrame(df_lig,'Ligand_smiles')
for i,j in Descriptors.descList:
    df_lig[i] = df_lig.ROMol.map(j)
df_lig = df_lig.add_prefix('L_')

df_l = df_lig.drop(columns=['L_Ligand_smiles', 'L_ROMol'])
print(df_l.shape)
df_2_l = remove_all_zero_col(df_l)
df_2_l = df_2_l.dropna(how='any', axis=1)
print(df_2_l.shape)
print(df_2_l)

Failed to patch pandas - unable to change molecule rendering


(372, 210)
(372, 140)
     L_MaxAbsEStateIndex  L_MaxEStateIndex  L_MinAbsEStateIndex  \
0               2.293853          2.293853             0.408678   
1               2.293853          2.293853             0.408678   
2               2.293853          2.293853             0.408678   
3               2.293853          2.293853             0.408678   
4               2.293853          2.293853             0.408678   
..                   ...               ...                  ...   
367            11.332358         11.332358             0.000000   
368             2.485828          2.485828             0.520842   
369             2.521862          2.521862             0.486508   
370             2.317770          2.317770             0.389499   
371             2.319721          2.319721             0.365852   

     L_MinEStateIndex     L_qed      L_SPS  L_MolWt  L_HeavyAtomMolWt  \
0           -0.408678  0.400516  11.037037  384.399           362.223   
1           -0.408678  0.40

In [6]:
# RDKit, Substrate
PandasTools.AddMoleculeColumnToFrame(df_sub,'Substrate_smiles')
for i,j in Descriptors.descList:
    df_sub[i] = df_sub.ROMol.map(j)
df_sub = df_sub.add_prefix('S_')

df_s = df_sub.drop(columns=['S_Substrate_smiles', 'S_ROMol'])
print(df_s.shape)
df_2_s = remove_all_zero_col(df_s)
print(df_2_s.shape)
print(df_2_s)

Failed to patch pandas - unable to change molecule rendering


(372, 210)
(372, 118)
     S_MaxAbsEStateIndex  S_MaxEStateIndex  S_MinAbsEStateIndex  \
0              11.846296         11.846296             0.075185   
1              12.068896         12.068896             0.083333   
2              12.163922         12.163922             0.013605   
3              12.644567         12.644567             0.250000   
4              10.645370         10.645370             0.120926   
..                   ...               ...                  ...   
367            11.285509         11.285509             0.311898   
368            11.285509         11.285509             0.311898   
369            11.285509         11.285509             0.311898   
370            11.285509         11.285509             0.311898   
371            11.285509         11.285509             0.311898   

     S_MinEStateIndex     S_qed      S_SPS  S_MolWt  S_HeavyAtomMolWt  \
0            0.075185  0.652473   9.714286  182.222           172.142   
1            0.083333  0.69

In [7]:
RDKit = pd.concat([Ligand_name, Ligand_smiles, df_2_l, Substrate_name, df_2_s, Yields], axis=1, join='inner')
print(RDKit)
RDKit.to_csv('../data/RDKit2.csv', index = False)

    Ligand_name                                      Ligand_smiles  \
0            L1  P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=CC=CC=C3)C4=C...   
1            L1  P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=CC=CC=C3)C4=C...   
2            L1  P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=CC=CC=C3)C4=C...   
3            L1  P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=CC=CC=C3)C4=C...   
4            L1  P(C1=CC=CC=C1)(C2=CC=CC=C2)CP(C3=CC=CC=C3)C4=C...   
..          ...                                                ...   
367         L98  O=S(C1=CC=CC(P(C2=CC=CC=C2)C3=CC=CC=C3)=C1)([O...   
368         L99  CC(P(C1=CC=C(N(C)C)C=C1)C2=CC=C(N(C)C)C=C2)CC(...   
369        L100  CC(P(C1=CC(C)=CC(C)=C1)C2=CC(C)=CC(C)=C2)CC(C)...   
370        L101                              CCCCCCP(CCCCCC)CCCCCC   
371        L102                        CCCCCCCCP(CCCCCCCC)CCCCCCCC   

     L_MaxAbsEStateIndex  L_MaxEStateIndex  L_MinAbsEStateIndex  \
0               2.293853          2.293853             0.408678   
1               2.293853 